# Reading an epub file and translating it using a transformer model

In [2]:
from epub_conversion.utils import open_book, convert_epub_to_lines
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import re, time
from tqdm.notebook import tqdm

# installing other libraries
# import sys
# !{sys.executable} -m pip install xml_cleaner

## get model

In [3]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-da",truncation=True, max_length=500)
model     = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-da")

## read book

In [4]:
book = open_book("/Users/peerchristensen/Downloads/DaVinciCode.epub")

lines = convert_epub_to_lines(book)

## clean text

In [5]:
def clean_text(text):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', text)
  return cleantext

cleaned_text = [clean_text(line) for line in lines]

def trim_text(s, n):
    return ' '.join(s.split()[:n])

trimmed_text = [trim_text(text,500) for text in cleaned_text]

## translate

In [6]:
translated = []

tic = time.perf_counter()

for i in tqdm(trimmed_text):
    try:
        input  = tokenizer.encode(i, return_tensors="pt")
        output = model.generate(input, early_stopping=True)
        trans  = tokenizer.decode(output[0])
        translated.append(trans)
    except:    
        pass

    
toc = time.perf_counter()

print(f"Time to translate: {(toc - tic) / 60:0.2f} minutes")


Time to translate: 3.27 minutes


## Clean translation

In [ ]:
clean_translation = [clean_text(text) for text in translated]

stripped = [text.strip() for text in clean_translation]

In [ ]:
stripped

## Save file

In [ ]:
file = open('davinci_translated_en_da2.txt','w')
for element in stripped:
    file.write(element+'\n')

file.close()